In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
from typing import Optional, List
from shapely.geometry import Polygon
from ipywidgets import interact, FloatSlider, Label
from dataclasses import dataclass
import os
import warnings

# Suppress the specific warning
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Exercise 2: **Backscattering Coefficient**

Visualize the backscattering coefficient (and its individual contributions) of a bare-
and vegetated soil-surface (in dB) for various soil and vegetation conditions within the angular range of $\theta \in [20\degree,75\degree]$ using an observation frequency of approx. 5 GHz.
The vegetative coverage should be modelled using the Cloud-Model as given in Eq. 6.64 of the
lecture notes. (Use reasonable values for $\tau$ and $\omega$ !).
For the bare-soil backscattering coefficient, a simple bare-soil model as proposed by Champion1
can be used which assumes that the bare-soil backscattering coefficient is linearly related to soil-
moisture in the microwave-domain, and it’s angular dependency can be described by a simple
cosine-model as given below:

$\sigma^0_{soil} = C\left(\theta\right) + D\cdot m_v$ [dB], with $C\left(\theta\right) = C_1 + C_2\cos^{C_3}\left(\theta\right)$.

The values for the parameters $C_1, C_2, C_3$ from Champion can be found in Fig.1. An estimate
on the parameter D which is usually referred to as the soil moisture sensitivity can be obtained
from the findings of Ulaby et.al shown in Fig. 2. Use the values where the measurement specifications fit best.


## 2.1 Is the consideration of vegetation effects necessary for measurements in the microwave domain?

-   inhomogeneities (e.g. vegetation components) give rise to _volume scattering_
-   in general, vegetation canopies ( = layers) are treated as volume scatterers
-   (in-)homogeneity in azimuthal vegetation density distribution gives rise to surface scattering contribution & scattering interactions between both/explains lack thereof
-   e.g. with C-band, dense forests are not transparent to MWs (masking out of SM)


## 2.2 How is a vegetation coverage treated within the Cloud-Model? (i.e. what simplifications are applied?)

-   vegetation is contained in a layer inbetween the soil/ground and atmsosphere
-   layer consists of discrete scateters in front of a uniform background
-   scatterers are identical (no differentiation between individual vegetation components)
-   spatial distribution of scatterers is random, though uniform across the layer (azimuthal and and polar directions)


## 2.3 What are the main effects of a vegetation coverage on the backscattering coefficient under the Cloud-Model approximation?

-   vegetation effects expressed through one parameter, the **extinction coefficient**:
    -   scattering coefficient $\kappa_s$
    -   absorption coefficient $\kappa_a$
    -   $\Rightarrow$ extinction coefficient $\kappa_e = \kappa_a + \kappa_s$
-   $\tau-\omega$ model contains two main paramaters:

    -   optical depth of canopy $\tau = \kappa_e \cdot d$
        -   $\tau \rightarrow 0$: bare soil, no attenuation
        -   $\tau \rightarrow \infty$: complete masking out of soil
    -   single-scattering albedo $\omega = \dfrac{\kappa_s}{\kappa_e}$

-   main effects are thus absorption and scattering $\Rightarrow$ attenuation of an incident signal by vegetation layer, i.e. scatterers contained

Thus, in the water cloud model, the backscttering coefficient is given as:

$\sigma^0\left(\theta\right) = \dfrac{3\omega\cos\left(\theta\right)}{4}\left(1-\gamma^2\right) + \gamma^2\sigma^0_{soil}$, with the attenuation factor $\gamma = \exp\left(-\dfrac{\tau}{\cos\theta}\right)$


In [2]:
@dataclass
class CContainer:
    pol: str
    c1: float
    c2: float
    c3: float


@dataclass
class DContainer:
    pol: str
    freq: float
    theta: float
    rms: float
    rho: float
    d: float

In [3]:
FREQ = 5e9  # Hz
THETA_0 = 40  # deg
THETAS = np.linspace(20, 75, 1000)  # deg
TAU = 0.01
OMEGA = 0.075
Vol_Water_Cont = 0.5

In [4]:
def db2lin(dB_value: float) -> float:
    return 10**(dB_value / 10)


def lin2db(linear_value: float) -> float:
    return 10 * np.log10(linear_value)


In [9]:
def sigma_soil(theta: float, c: CContainer, d: float, m_v: float) -> float:
    '''Calculates the backscatter coefficient of the soil, based on Champion model.

    Parameters
    ----------
    theta : float
        Angle of incidence in rad.
    c : CContainer
        Contains the constants for the soil type accoding to the Champion model.
    d : float
        Soil moisture sensitivity.
    m_v : float
        Soil volumetric moisture content m^3/m^3.

    Returns
    -------
    float
        Backscatter coefficient in dB.
    '''
    return c.c1 + c.c2 * np.cos(theta)**c.c3 + d * m_v


def sigma_canopy(theta: float, omega: float) -> float:
    return (3 * omega * np.cos(theta)) / (4)


def sigma_0(theta: float, c: CContainer, d: float, m_v: float, omega: float,
            tau: float) -> float:
    '''Calculates the resulting backscatter coefficient sigma0.

    Parameters
    ----------
    theta : float
        Angle of incidence in rad.
    c : CContainer
        Contains the constants for the soil type accoding to the Champion model.
    d : float
        Soil moisture sensitivity.
    m_v : float
        Soil volumetric moisture content m^3/m^3.
    omega : float
        Single-scattering albedo.
    tau : float
        Optical density.

    Returns
    -------
    float
        Backscatter coefficient in dB.'''
    gamma = np.exp(-tau / np.cos(theta))
    return sigma_canopy(theta, omega) * (1 - gamma**2) + gamma**2 * db2lin(
        sigma_soil(theta, c, d, m_v))
    return lin2db(
        sigma_canopy(theta, omega) * (1 - gamma) +
        gamma**2 * db2lin(sigma_soil(theta, c, d, m_v)))

In [10]:
HH_5_3 = CContainer(pol='HH', c1=-29.21, c2=27.2, c3=2.8)
VV_5_3 = CContainer(pol='VV', c1=-26.0, c2=24.0, c3=2.7)
HV_5_3 = CContainer(pol='HV', c1=-33.0, c2=16.0, c3=3.2)

In [11]:
D_HH_20_1_5_4_1 = DContainer(pol='HH',
                             theta=20.0,
                             freq=1.5,
                             rms=4.1,
                             rho=0.87,
                             d=0.17)
D_HH_20_1_5_1_1 = DContainer(pol='HH',
                             theta=20.0,
                             freq=1.5,
                             rms=1.1,
                             rho=0.82,
                             d=0.24)
D_HH_10_4_25_4_1 = DContainer(pol='HH',
                              theta=10.0,
                              freq=4.25,
                              rms=4.1,
                              rho=0.9,
                              d=0.28)
D_HH_10_4_25_1_1 = DContainer(pol='HH',
                              theta=10.0,
                              freq=4.25,
                              rms=1.1,
                              rho=0.9,
                              d=0.32)

In [12]:
omega_slider = FloatSlider(value=OMEGA,
                           min=0,
                           max=1,
                           step=0.01,
                           description=r'omega:')
tau_slider = FloatSlider(value=TAU, min=0, max=5, step=0.1, description='tau:')

m_slider = FloatSlider(value=Vol_Water_Cont,
                       min=0,
                       max=1,
                       step=0.05,
                       description='m_v [m^3/m^3]:')


@interact(omega=omega_slider, tau=tau_slider, m_v=m_slider)
def update(omega, tau, m_v):
    fig = plt.figure(figsize=(12, 6))
    plt.style.use([
        # os.path.join('custom_style.mplstyle'),
        'science',
        # 'ieee',
    ])
    ax1 = fig.add_subplot(121)
    thetas = np.deg2rad(THETAS)

    def plot(
        ax,
        d,
    ):
        ax.plot(
            thetas,
            sigma_0(thetas, HH_5_3, d.d, m_v, omega, tau),
            linewidth=2,
            label=
            fr'$D\left(\nu={d.freq:.2f}GHz, \theta={np.deg2rad(d.theta):.2f}rad, RMS={d.rms:.2f}cm\right)$'
        )

    plot(ax1, D_HH_20_1_5_4_1)
    plot(ax1, D_HH_20_1_5_1_1)
    plot(ax1, D_HH_10_4_25_4_1)
    plot(ax1, D_HH_10_4_25_1_1)

    ax1.legend()
    ax1.set_xlabel(r'$\theta$ [rad]', fontsize=12)
    ax1.set_ylabel(r'$\sigma^0$', fontsize=12)

    ax2 = fig.add_subplot(122)

    def plot_db(
        ax,
        d,
    ):
        ax.plot(
            thetas,
            lin2db(sigma_0(thetas, HH_5_3, d.d, m_v, omega, tau)),
            linewidth=2,
            label=
            fr'$D\left(\nu={d.freq:.2f}GHz, \theta={np.deg2rad(d.theta):.2f}rad, RMS={d.rms:.2f}cm\right)$'
        )

    plot_db(ax2, D_HH_20_1_5_4_1)
    plot_db(ax2, D_HH_20_1_5_1_1)
    plot_db(ax2, D_HH_10_4_25_4_1)
    plot_db(ax2, D_HH_10_4_25_1_1)

    ax2.legend()
    ax2.set_xlabel(r'$\theta$ [rad]', fontsize=12)
    ax2.set_ylabel(r'$\sigma^0$ [dB]', fontsize=12)

    # ax.set_xlim(-1010, 1010)
    fig.suptitle(
        fr'$\omega$: {omega:.2f}, $\tau$: {tau:.2f}, $m_v$: {m_v:.2f} $m^3/m^3$',
        fontsize=16)

    # plt.tight_layout()
    plt.show()

interactive(children=(FloatSlider(value=0.075, description='omega:', max=1.0, step=0.01), FloatSlider(value=0.…